In [0]:
!pip install paddlepaddle
# !pip install paddlepaddle-gpu==0.14.0.post87
# !export LD_LIBRARY_PATH="$LD_LIBRARY_PATH:/usr/local/lib/:/usr/local/lib/libmkldnn.so.0:/usr/local/lib/libmklml_intel.so"
!ldconfig
# !export LD_LIBRARY_PATH="$LD_LIBRARY_PATH:/usr/local/lib"
import contextlib
from __future__ import print_function, division
import json
from PIL import Image
import numpy as np
import random
import paddle.v2 as paddle
import paddle.fluid as fluid
import paddle.fluid.framework as framework
import paddle.fluid.layers as pd
from paddle.fluid.executor import Executor

In [0]:
"""Unzip data"""

!tar -xf baidu_star_2018_train_stage1_new.tar
!mkdir baidu_star_2018/image/preprocessed
!ls baidu_star_2018/

In [0]:
"""Data pre-processing

    An image is resized to a certain height and split into many ribbons with the same width.
    In a ribbon, any appearing person is assigned to a corresponding slot.
    There is a label for every ribbon. Elements from the lower half of the label indicate 
    the probability of containing at least one target in that slot, while the high half indicates
    the number of targets in the slot.

    'train_names' and 'test_names' store names of split, processed ribbon files
    'data_dict' stores labels keyed by file names
    

"""

from __future__ import print_function, division
import pickle
import numpy as np
import json
import random
import math
import matplotlib.pyplot as plt
import cv2
import time
from PIL import Image, ImageFilter, ImageOps, ImageEnhance
from IPython.display import HTML, display

height = 640
crop_width = 112
stride = int(crop_width/2)
label_slots = 10
update_data = True

start_time = time.time()

with open('baidu_star_2018/annotation/annotation_train_stage1.json', 'r') as json_f:
    annotations_array = json.load(json_f)['annotations']
random.shuffle(annotations_array)
for item in annotations_array:
  item['name'] = item['name'][13:]
train_names = []
test_names = []

if update_data:
  data_dict = {}
def process_data(anarr):
  def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))
  print('processing data')
  out = display(progress(0, 100), display_id=True)
  prog = 0
  length_a = len(anarr)
  space = int(round((crop_width - stride)/2))
  for item in anarr:
    img = Image.open('baidu_star_2018/image/stage1/train/'+item['name'])
    img = np.asarray(img)
    for region in item['ignore_region']:
      ignore_points = []
      for point in region:
          ignore_points.append([point['x'], point['y']])
      cv2.fillPoly(img, np.array([ignore_points]), np.mean(img,axis=(0,1)))
    img = Image.fromarray(img)
    w,h = img.size
    scale = height/h
    img = img.resize((int(scale*w), int(scale*h)+1),Image.ANTIALIAS)
    anns = []
    for ann in item['annotation']:
      x = int(scale*ann['x'])
      y = int(scale*ann['y'])
      if item['type'] == 'bbox':
        pw = scale*ann['w']
        ph = scale*ann['h']
        x = x + int(pw/2)
        y = y + int(0.1*ph)
      anns.append((x,y))
    w2,h2 = img.size
    sw = int(w2/stride)
    
    modalities = []
    modalities.append(img)
    modalities.append(ImageEnhance.Color(img).enhance(random.randint(1,10)/10))
    
    for mod in range(len(modalities)):
      mod_img = modalities[mod]

      rands = random.randint(stride,int(crop_width*0.8))
      for s in range(sw):
        shift = stride*s - rands
        box = (shift, 0, shift+crop_width, height)
        mid_box = (space, 0, space+stride, height)
        crop_anns = []
        label = [0,0] * label_slots
        for ann2 in anns:
          if ann2[0] >= shift + space and ann2[0] < shift + crop_width - space:
            y = ann2[1]
            pos = int(label_slots*y/height)
            label[pos] = 1
            label[pos+10] += 1
        img_crop = mod_img.crop(box)
        for mirror in ['m','o']:
          for flip in ['f', 'p']:
            img_crop_mf = img_crop
            img_crop_label = np.array(label)
            if mirror == 'm':
              img_crop_mf = ImageOps.mirror(img_crop_mf)
            if flip == 'f':
              img_crop_mf = ImageOps.flip(img_crop_mf)
              img_crop_label = np.reshape(img_crop_label, [-1,2])
              img_crop_label = np.flip(img_crop_label, 0)
              img_crop_label = np.reshape(img_crop_label, [-1])

            crop_name = str(s) + 'm' + str(mod) + mirror + flip + item['name']
            data_dict[crop_name] = img_crop_label
            """darken two sides"""
            img_crop_mid = img_crop_mf.crop(mid_box)
            img_crop_mf = img_crop_mf.point(lambda p: p*0.6)
            img_crop_mf.paste(img_crop_mid, mid_box)

            img_crop_mf.save('baidu_star_2018/image/preprocessed/'+crop_name)
            if prog < length_a * 0.85:
              train_names.append(crop_name)
            else:
              test_names.append(crop_name)
        
    if prog % 17 ==0:
      percent =  (prog+1) / length_a * 100
      out.update(progress(percent, 100))
    prog += 1
if update_data:
  process_data(annotations_array)
  elapsed_time = time.time() - start_time
  print('finished in', str(round(elapsed_time,2))+'s')
annotations_array = None
random.shuffle(train_names)
random.shuffle(test_names)
import pickle
with open('data_dict.pkl', 'wb') as f:
  pickle.dump([data_dict, train_names, test_names], f)
print('data: ', len(data_dict))
print('training data: ', len(train_names))
print('test data: ', len(test_names))


In [0]:
"""Print a ribbon"""

img = Image.open('baidu_star_2018_test_stage1/baidu_star_2018/image/preprocessed/'+'2a2c372ac00bd4c3e9dc683b7969c07bd.jpg')
img = np.asarray(img)
plt.imshow(img)
plt.axis('off')
plt.show()
print(test_data_dict['2a2c372ac00bd4c3e9dc683b7969c07bd.jpg'])

In [0]:
"""Hyper parameters"""
batch_size = 100
l2_lambda = 0.0001
beta1 = 1
beta2 = 1
beta3 = 2
alpha = 1e-5

In [0]:
"""Training data batches"""

def MaxMinNormalization(x):
      Min = np.min(x)
      Max = np.max(x)
      if Max != Min:
        return (x - Min) / (Max - Min)
      else:
        return x
def train_data():
  for name in train_names:
    img = Image.open('baidu_star_2018/image/preprocessed/'+name)
    img = np.asarray(img)
    img = MaxMinNormalization(img)
    label = data_dict[name]
    yield img, label
def test_data():
  for name in test_names:
    img = Image.open('baidu_star_2018/image/preprocessed/'+name)
    img = np.asarray(img)
    img = MaxMinNormalization(img)
    label = data_dict[name]
    yield img, label
    
train_reader = paddle.batch(
    train_data,
    batch_size=batch_size)

test_reader = paddle.batch(
    test_data,
    batch_size=batch_size)

In [9]:
"""Define the model

    This model has a inception structure, though its configuration is a lot 
    different from the one introduced in the GoogLeNet paper.

"""

def conv(kernel_size, input, filters, padding=0, strides=(1,1)):
  return pd.conv2d(input, filters, kernel_size,
                          stride = strides,
                          act='relu',
                          padding=padding,
                          param_attr=fluid.initializer.Xavier(),
                          bias_attr=fluid.initializer.Constant(0.0))
def dense(input, output_size, act='sigmoid'):
  return pd.fc(input, output_size,
                          act=act,
                          param_attr=fluid.initializer.Xavier(),
                          bias_attr=fluid.initializer.Constant(0.0))
  
  
def inception_unit(input, output_size):
    block_size = int(output_size/4)
    
    incep_1_conv_1x1_1 = conv(1, input, block_size)

    incep_1_conv_1x1_2 = conv(1, input, block_size)

    incep_1_conv_1x1_3 = conv(1, input, block_size)

    incep_1_max_p = pd.pool2d(input, pool_size=3, pool_stride=1, pool_padding=1)

    incep_1_conv_1x1_4 = conv(1, incep_1_max_p, block_size)

    incep_1_conv_3x3 = conv(3, incep_1_conv_1x1_2, block_size, padding=1)
    
    incep_1_conv_5x5 = conv(5, incep_1_conv_1x1_3, block_size, padding=2)

    concat = pd.concat([incep_1_conv_1x1_1, incep_1_conv_1x1_4, incep_1_conv_3x3, incep_1_conv_5x5], 1, name='inception_unit')

    return concat
  
def conv_pool(input, strides=(2,2)):
  block_size = int(input.shape[-3]/2)

  conv_pool_conv_1x1_1 = conv(1, input, block_size)
  
  conv_pool_conv_1x1_2 = conv(1, input, block_size)
  
  conv_pool_conv_3x3_1 = conv(3, conv_pool_conv_1x1_1, block_size, padding=1)
  
  conv_pool_conv_3x3_2 = conv(3, conv_pool_conv_1x1_2, block_size, strides=strides, padding=1)
  
  conv_pool_conv_3x3_3 = conv(3, conv_pool_conv_3x3_1, block_size, strides=strides, padding=1)
  
  max_pool = pd.pool2d(input, pool_size=3, pool_stride=strides, pool_padding=(1,1))
  
  concat = pd.concat([conv_pool_conv_3x3_2, conv_pool_conv_3x3_3, max_pool], 1, name='conv_pool')

  return concat

def inception_cnn(input, is_test):

  a = conv(3, input, 16, padding=1)
  print('1', a.shape)
  bn = pd.batch_norm(a, is_test=is_test)
  a = conv(3, a, 32, padding=1)
  print('2', a.shape)
  bn = pd.batch_norm(a, is_test=is_test)
  a = conv_pool(a, (2,2))
  print('3', a.shape)
  a = inception_unit(a, 64)
  print('4', a.shape)
  bn = pd.batch_norm(a, is_test=is_test)
  a = conv_pool(a, (2,2))
  print('5', a.shape)
  a = inception_unit(a, 64)
  print('6', a.shape)
  bn = pd.batch_norm(a, is_test=is_test)
  a = conv_pool(a, (2,2))
  print('7', a.shape)
  a = inception_unit(a, 128)
  print('8', a.shape)
  bn = pd.batch_norm(a, is_test=is_test)
  a = conv_pool(a, (2,2))
  print('9', a.shape)
  a = inception_unit(a, 128)
  print('10', a.shape)
  bn = pd.batch_norm(a, is_test=is_test)
  a = conv_pool(a, (2,1))
  print('11', a.shape)
  a = inception_unit(a, 128)
  print('12', a.shape)
  bn = pd.batch_norm(a, is_test=is_test)
  a = conv_pool(a, (2,1))
  print('13', a.shape)
  a = inception_unit(a, 128)
  print('14', a.shape)
  a = conv_pool(a, (2,1))
  print('15', a.shape)
  d = dense(a, 2048, act='leaky_relu')
  d = pd.dropout(d, 0.5, is_test=is_test)
  d = dense(d, 1024)
  print('16', d.shape)
  return d

def model(is_test=False):
    """input: [batch_size, channels, width, height]"""

    input = fluid.layers.data(name='input', shape=[3, height, crop_width], dtype='float32')
    
    cnn_output = inception_cnn(input, is_test)
    print('cnn output:', cnn_output.shape)

    pred_nums_0 = dense(cnn_output, 256)
    pred_nums = dense(pred_nums_0, label_slots, act=None) # linear
    
    pred_probs_0 = dense(cnn_output, 256)
    pred_probs = dense(pred_probs_0, label_slots) # logistic

    return pred_probs, pred_nums

def train_program():
    labels = fluid.layers.data(name='labels', shape=[1], dtype='float32')
    
    pred_probs, pred_nums = model()
  
    label_nums = pd.slice(labels, axes = [0, 1], starts=[0,label_slots], ends=[9999,9999])
    num_loss = pd.reduce_mean(pd.reduce_sum(pd.square_error_cost(label_nums, pred_nums), dim=1))

    label_probs = pd.slice(labels, axes = [0, 1], starts=[0,0], ends=[9999,label_slots])
    pred_probs = pd.clip(pred_probs, min=1e-10, max=1- (1e-10))
    probs_loss = pd.reduce_mean(pd.reduce_sum((-1) * label_probs * pd.log(pred_probs) - (1-label_probs) * pd.log(1 - pred_probs), dim=1 ))

    count_loss = pd.reduce_mean(pd.reduce_sum(pd.abs(label_probs*label_nums - pred_probs*pred_nums), dim=1))
    
    loss = num_loss * beta1 + probs_loss * beta2 + count_loss * beta3
    
    return loss
  
def optimizer_program():
    return fluid.optimizer.Adam(learning_rate=alpha)

use_cuda = False
place = fluid.CUDAPlace(0) if use_cuda else fluid.CPUPlace()

trainer = fluid.Trainer(
    train_func=train_program,
    place=place,
    optimizer_func=optimizer_program)
feed_order=['input', 'labels']

# Specify the directory to save the parameters
params_dirname = "a.model"

# Plot data
from paddle.v2.plot import Ploter
train_title = "Train cost"
test_title = "Test cost"
plot_cost = Ploter(train_title, test_title)

step = 0

# event_handler prints training and testing info
def event_handler_plot(event):
    global step
    if isinstance(event, fluid.EndStepEvent):
        if event.step % 50 == 0: #record a test cost every 10 batches
            test_metrics = trainer.test(
                reader=test_reader, feed_order=feed_order)

            plot_cost.append(test_title, step, test_metrics[0])
            plot_cost.plot()

            if test_metrics[0] < 0.1:
                # If the accuracy is good enough, we can stop the training.
                print('loss is less than 0.1, stop')
                trainer.stop()

        # We can save the trained parameters for the inferences later
        if params_dirname is not None:
            trainer.save_params(params_dirname)

        step += 1

1 (-1L, 16L, 640L, 112L)
2 (-1L, 32L, 640L, 112L)
3 (-1L, 64L, 320L, 56L)
4 (-1L, 64L, 320L, 56L)
5 (-1L, 128L, 160L, 28L)
6 (-1L, 64L, 160L, 28L)
7 (-1L, 128L, 80L, 14L)
8 (-1L, 128L, 80L, 14L)
9 (-1L, 256L, 40L, 7L)
10 (-1L, 128L, 40L, 7L)
11 (-1L, 256L, 20L, 7L)
12 (-1L, 128L, 20L, 7L)
13 (-1L, 256L, 10L, 7L)
14 (-1L, 128L, 10L, 7L)
15 (-1L, 256L, 5L, 7L)
16 (-1L, 1024L)
cnn output: (-1L, 1024L)


In [0]:
"""Training"""

trainer.train(
    reader=train_reader,
    num_epochs=10,
    event_handler=event_handler_plot,
    feed_order=feed_order)

In [0]:
"""Testing"""

inferencer = fluid.Inferencer(
    infer_func=model,
    param_path=params_dirname,
    place=place)

In [0]:
"""unzip test data"""

!unzip baidu_star_2018_test_stage1.zip
!ls baidu_star_2018_test_stage1/baidu_star_2018
!mkdir baidu_star_2018_test_stage1/baidu_star_2018/image/preprocessed

In [0]:
"""Preparing data for test"""

from __future__ import print_function, division
import pickle
import numpy as np
import json
import random
import math
import time
import matplotlib.pyplot as plt
import cv2
from PIL import Image, ImageFilter
from IPython.display import HTML, display

height = 640
crop_width = 112
stride = int(crop_width/2)
update_data = True

start_time = time.time()

with open('baidu_star_2018_test_stage1/baidu_star_2018/annotation/annotation_test_stage1.json', 'r') as json_f:
    annotations_array = json.load(json_f)['annotations']

for item in annotations_array:
  item['name'] = item['name'][12:]

if update_data:
  test_data_dict = {}
  test_data_names = []
def process_data(anarr):
  def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))
  print('processing data')
  out = display(progress(0, 100), display_id=True)
  prog = 0
  length_a = len(anarr)
  space = int(round((crop_width - stride)/2))
  for item in anarr:
    img = Image.open('baidu_star_2018_test_stage1/baidu_star_2018/image/stage1/test/'+item['name'])
    
    img = np.asarray(img)
    for region in item['ignore_region']:
      ignore_points = []
      for point in region:
          ignore_points.append([point['x'], point['y']])
      cv2.fillPoly(img, np.array([ignore_points]), np.mean(img,axis=(0,1)))
    img = Image.fromarray(img)
    
    w,h = img.size
    scale = height/h
    img = img.resize((int(scale*w), int(scale*h)+1),Image.ANTIALIAS)

    w2,h2 = img.size
    sw = int(w2/stride)
    rands = random.randint(stride,int(crop_width*0.8))
    
    for s in range(sw):
      shift = stride*s - rands
      box = (shift, 0, shift+crop_width, height)
      mid_box = (space, 0, space+stride, height)
      img_crop = img.crop(box)
      """darken two sides"""
      img_crop_mid = img_crop.crop(mid_box)
      img_crop = img_crop.point(lambda p: p*0.6)
      img_crop.paste(img_crop_mid, mid_box)
      crop_name = str(s)+item['name']
      img_crop.save('baidu_star_2018_test_stage1/baidu_star_2018/image/preprocessed/'+crop_name)
      test_data_dict[crop_name] = {
          'label': [],
          'id': item['id']
      }
      test_data_names.append(crop_name)
    if prog % 17 ==0:
      percent =  (prog+1) / length_a * 100
      out.update(progress(percent, 100))
    prog += 1
if update_data:
  process_data(annotations_array)
  elapsed_time = time.time() - start_time
  print('finished in', str(round(elapsed_time,2))+'s')
annotations_array = None
import pickle
with open('test_data_dict.pkl', 'wb') as f:
  pickle.dump([test_data_dict, test_data_names], f)
print('data: ', len(test_data_dict))
print('test data: ', len(test_data_names))

In [0]:
"""Function for reading a test batch"""

current_batch = 0
def MaxMinNormalization(x):
  Min = np.min(x)
  Max = np.max(x)
  if Max != Min:
    return (x - Min) / (Max - Min)
  else:
    return x
def next_batch(batch_size):
  names = []
  global current_batch
  cb = current_batch
  if current_batch == -1:
    print('!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!epoch finished!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
    return None
  if cb+batch_size > len(test_data_names):
    current_batch = cb + batch_size - len(test_data_names)
    names = test_data_names[cb : len(test_data_names)] + test_data_names[0: current_batch ]
    current_batch = -1
  else:
    names = test_data_names[cb : cb+batch_size]
    current_batch = cb+batch_size
  images = []
  for name in names:
    img = Image.open('baidu_star_2018_test_stage1/baidu_star_2018/image/preprocessed/'+name)
    img = np.asarray(img)
    img = MaxMinNormalization(img)
    images.append(img)
  return np.array(images), names

In [0]:
"""Run tests"""

while True:
  batch = next_batch(100)
  if batch is not None:
    images, names = batch
    test_predict = inferencer.infer({'input': images})
    test_predict = np.stack(test_predict,axis=1)
    test_predict = np.transpose(test_predict, [0,2,1])
    test_predict = np.reshape(test_predict, [batch_size, -1])
    for m in range(len(names)):
      test_data_dict[names[m]]['label'] = test_predict[m]
  else:
    break


In [0]:
"""Construct results"""
results = {}
for name,item in test_data_dict.items():
  predicted_label = item['label']
  its_id = item['id']
  num = 0
  for i in range(0, len(predicted_label), 2):
    pred_num = predicted_label[i+1]
    num += pred_num
  if results.has_key(its_id):
    results[its_id] += num
  else:
    results[its_id] = num
for key in results:
  results[key] = int(round(results[key]))
len(results)

In [0]:
"""Get results"""

import csv
with open('results.csv', 'wb') as csvfile:
    rcsv = csv.writer( csvfile, dialect='excel' )
    rcsv.writerow(['id', 'predicted'])
    for (k,v) in results.items():
      rcsv.writerow([k, v])
      
from google.colab import files
files.download('results.csv')